# Исследование надежности заемщиков

**Цель исследования:** определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

**Ход исследования:**
 1. Обзор данных.
 2. Предобработка данных.
 3. Расчёты и добавление результатов в таблицу
 4. Ответы на вопросы.
 5. Общий вывод.

 **Данные**
 
 Данные клиентов банка:
- Количество детей в семье
- Общий трудовой стаж в днях
- Возраст клиента в годах
- Уровень образования клиента
- Идентификатор уровня образования
- Семейное положение
- Идентификатор семейного положения
- Пол клиента
- Тип занятости
- Имел ли задолженность по возврату кредитов
- Ежемесячный доход
- Цель получения кредита
 
 **Источник данных:** статистика о платёжеспособности клиентов банка.

## Обзор данных

In [4]:
import pandas as pd # импорт библиотеки pandas
df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в customers_data
df.info() # получение общей информации о данных в таблице customers_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Таблица с данными имеет 21525 строк и 12 столбцов.

Данные в таблице имеют типы int64, float64, object.

В столбцах days_employed и total_income есть незаполненные ячейки.

Названия колонок выполнены корректно и не требуют обработки.

In [5]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбце days_employed есть некорректные (отрицательные) значения.

## Предобработка данных

### Заполнение пропусков

In [6]:
df.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход) одинаковое количество пропусков - 2174. Исходя из этого, можно предположить, что в 2174 строках не заполнены ячейки двух столбцов. Есть вероятность, что это связано с проблемой при сброе данных - необходимо сообщить сотруднику, отвечающему за это.

Т.к. данные пропущены в столбцах, которые будут использоваться для анализа, необходимо заполнить недостающие данные.

In [7]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median')) #замена пропущенных значений в столбце total_income на медианное значение total_income для группы, к которой заемщик относится по типу занятости
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median')) #замена пропущенных значений в столбце days_employed на медианное значение days_employed для группы, к которой заемщик относится по типу занятости

Проверим, все ли пропуcки заполнены

In [8]:
df.isna().sum() # подсчёт пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропуски запонены, переходим к следующему этапу предобработки

### Проверка данных на аномалии и исправления.

**Аномалии в столбце children**

Проверим на аномалии столбец children (количество детей в семье), т.к. в исследовании поставлен вопрос, для которго постребуются данные этого столбца. Выведем все уникальные значения этого столбца и посчитаем количество строк с ними в таблице.

In [9]:
print(df.groupby('children')['children'].count())

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64


В списке есть 2 аномальных значения: 

-1 - число детей не может быть отрицательным;

20 - в целом, возможное значение, но с учетом количества таких семей в выборке, можно предположить, что в данном случае это некорректные данные; скорее всего должно быть указано 2 ребенка.

Произведем замену аномальных значений: -1 заменим на 1, 20 на 2.

In [10]:
df['children'] = df['children'].replace({20: 2, -1: 1})

Проверим выполненные замены

In [11]:
print(df.groupby('children')['children'].count())

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


**Аномалии в столбце days_employed**


При певоначальном ознакомлении с датасетом в столбце days_employed были замечены отрицательные значения. Вероятнее всего, данные по стажу собраны некорректно. Приведем все отрицательные значения стажа к положительным.

In [12]:
df['days_employed'] = df['days_employed'].abs()
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Отрицательные значения преобразованы в положительные.

Так же в данных по стажу имеются значения, сильно превышающие возможный трудовой стаж. Рассчитаем значение максимального стажа и проверим, какая доля значений в таблице превышает его.

In [13]:
max_days_employed = (df['dob_years'].max() - 18) * 365 # вычитаем из максимального возраста заемщика возраст начала трудовой деятельности и умножаем на количество дней в году
print(df[df['days_employed'] > max_days_employed]['days_employed'].count() / len(df)*100) #находим процент значений стажа, превышающего максимальный

17.923344947735192


17,87% заемщиков в таблице имеют стаж больше максимально возможного. Это указывает на явные проблемы с заполненеим стажа при сборе данных. Т.к. для решения поставленных в задаче вопросов нет необходимости использовать данные стажа, на текущий момент не требуется дополнительно обрабатывать данные этого столбца. Но необходимо передать информацию об этой проблеме коллегам, собирающим данные. 

### Изменение типов данных.

In [14]:
df['total_income'] = df['total_income'].astype('int') #замена типа данных в столбце total_income с float64 на int64
df.info() # получение общей информации о данных в таблице customers_data для проверки замены типа данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Тип данных в столбце total_income (ежемесячный доход) изменен. Тип данных int64 больше подходит для данных о доходах, т.к. даходы измеряются в единицах, не имеющих большого количества знаков после запятой. Так же замена типа данных упрощает восприятие данных в таблице.

### Удаление дубликатов.

In [15]:
print('Количество явных дубликатов:', df.duplicated().sum()) # подсчёт явных дубликатов

Количество явных дубликатов: 54


In [16]:
df = df.drop_duplicates().reset_index() # удаление явных дубликатов (с удалением старых индексов и формированием новых)
print('Количество явных дубликатов:', df.duplicated().sum()) # подсчёт явных дубликатов

Количество явных дубликатов: 0


Проверим столбец education данными на неявные дубликаты.

In [17]:
df['education'].sort_values().unique() #вывод уникальных значений для столбца education

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Т.к. дубликаты отличаются только регистром, их можно обработать при помощи метода str.lower().

In [18]:
df['education'] = df['education'].str.lower()
df['education'].sort_values().unique() #проверка обработки

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

Неявные дубликаты в столбце education удалены.

**Проверим остальные столбцы с категориаольными значениями**

In [19]:
print(df['education_id'].sort_values().unique())
print(df['family_status'].sort_values().unique())
print(df['family_status_id'].sort_values().unique())
print(df['gender'].sort_values().unique())
print(df['income_type'].sort_values().unique())
print(df['purpose'].sort_values().unique())

[0 1 2 3 4]
['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']
[0 1 2 3 4]
['F' 'M' 'XNA']
['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудник' 'студент']
['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости' 'покупка недвижимости'
 'покупка своего жилья' 'получение высшего образования'
 'получение дополнительного образования' 'получение образования'
 'приобретение автомобиля' 'профильное образование' 'ремонт жилью'
 '

**Вывод**

Проверенные столбцы (кроме purpose) не содерджат неявных дубликатов.

Неявные дубликаты в столбце purpose удалять не будем - в шаге 2.7 разнесем цели по более крупным группам и запишем эту информацию в новый столбец.

## Расчёты и добавление результатов в таблицу

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [20]:
education_dict = df[['education_id', 'education']] #сформируем новый датафрейм из двух столбцов исходного
education_dict = education_dict.drop_duplicates().reset_index(drop=True) #удалим дубликаты в таблице education_dict
print(education_dict.sort_values('education_id')) #распечатаем support_dict в порядке возрастания значений столбца education_id

   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень


В результате мы получили словарь типов образования, к которому можно обращаться по идентификатору

Создадим аналогичный словарь для статусов семейного положения. 

In [21]:
family_status_dict = df[['family_status_id', 'family_status']] #сформируем новый датафрейм из двух столбцов исходного
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True) #удалим дубликаты в таблице family_status_dict
print(family_status_dict.sort_values('family_status_id')) #распечатаем support_dict в порядке возрастания значений столбца education_id

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем


Согласно заданию, столбцы education и family_status необходимо удалить из исходного датафрейма. Но для ответа на вопросы, поставленные в работе, будет удобнее использовать данные из этих столбцов, если они останутся в таблице. Ниже приведено решенте по удалению стобцов в закомментированом виде. 

In [22]:
#df = df.drop(['education', 'family_status'], axis = 1)

### Категоризация дохода.

Напишем функцию, добавляющую новый столбец в таблицу с категориями в зависимости от величины дохода:

0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'

In [23]:
def total_income_group(income): #функция, возвращающая категорию в зависимости от величины дохода
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    else:
        return 'A'

Протестируем работу функции для каждого правила

In [24]:
print(total_income_group(30000))
print(total_income_group(40000))
print(total_income_group(100000))
print(total_income_group(800000))
print(total_income_group(2000000))

E
D
C
B
A


Функция работает корректно

Добавим в таблицу столбец с результатами работы функции применительно к столбцу total_income

In [25]:
df['total_income_category'] = df['total_income'].apply(total_income_group)
display(df.head(5))

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


**Вывод**

Функция работает корректно, в таблицу добавлен новый столбец с категориями заемщиков в зависимости от их ежемесячного дохода. 

### Категоризация целей кредита.

<a id='section_id'></a>

Рассотрим значения столбца purpose

In [26]:
print(df['purpose'].sort_values().unique())

['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости' 'покупка недвижимости'
 'покупка своего жилья' 'получение высшего образования'
 'получение дополнительного образования' 'получение образования'
 'приобретение автомобиля' 'профильное образование' 'ремонт жилью'
 'свадьба' 'свой автомобиль' 'сделка с автомобилем'
 'сделка с подержанным автомобилем' 'строительство жилой недвижимости'
 'строительство недвижимости' 'строительство собственной недвижимости'
 'сыграть свадьбу']


Для похожих или одинаковых целей в стоблце purpose заполнены разные значения. Для упрощения анализа необходимо привести значения столбца purpose к нескольким категориям, объединяющим все значения с аналогичными целями.
Для этого используем вхождение строки в значение столбца.

In [27]:
def purpose_category_group(purpose): #функция, возвращающая категорию в зависимости от величины дохода
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'недв' in purpose:
        return 'операции с недвижимостью'
    if 'жил' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'

Протестируем работу функции для каждого правила

In [28]:
print(purpose_category_group('на покупку своего автомобиля'))
print(purpose_category_group('высшее образование'))
print(purpose_category_group('жилье'))
print(purpose_category_group('операции со своей недвижимостью'))
print(purpose_category_group('сыграть свадьбу'))

операции с автомобилем
получение образования
операции с недвижимостью
операции с недвижимостью
проведение свадьбы


Функция работает корректно

Добавим в таблицу столбец с результатами работы функции применительно к столбцу purpose

In [29]:
df['purpose_category'] = df['purpose'].apply(purpose_category_group)
display(df.head(5))

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на данный вопрос найдем среднее значение для столбца debt (оно отображает долю кредитов с задолженностью по всей выборке) и переведем его в проценты. Сравним полученное значение со средними значениями debt для категорий заемщиков с разным количеством детей. Так же посчитаем, количество заемщиков в каждой группе.

In [30]:
average_debt = df['debt'].mean()
print(average_debt)
print(df.groupby('children')['debt'].agg({'count', 'mean'}))

0.08108611615667645
          count      mean
children                 
0         14107  0.075353
1          4856  0.091639
2          2128  0.094925
3           330  0.081818
4            41  0.097561
5             9  0.000000


##### Вывод 1:

Средняя доля кредитов с задолженностью по выборке - 8,1%.

Для заемщиков без детей средняя доля кредитов с задолженностью составляет 7,53%, т.е. ниже, чем для любой группы заемщиков с детьми.

Для заемщиков с детьми доля кредитов с задолженностью составляет от 8,18% (3 ребенка) до 9,75% (4 ребенка).

По результатам расчета нельзя сказать, что с ростом числа детей в семье растет риск задолженности по кредиту, т.к. для семей с 3 детьми он ниже, чем для семей с 1 и 2 детьми. Но в то же время, максимальный процент невозврата показала группа с 4 детьми - 9,75%.
Есть вероятность, что в нашей выборке недостаточно представлена группа заемщиков с 3 детьми (1,44%) и доля доля кредитов с задолженностью, показанная для нее, не соответствует реальной, т.к. для семей с 1, 2 и 4 детьми показан рост доли кредитов с задолженностью в зависимости от количества детей.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на данный вопрос сравним среднее значение для столбца debt (оно отображает долю кредитов с задолженностью по всей выборке) со средними значениями debt для категорий заемщиков с разным семейным положением. Так же посчитаем, количество заемщиков в каждой группе.

In [31]:
print(average_debt)
print(df.groupby('family_status')['debt'].agg({'count', 'mean'}))

0.08108611615667645
                       count      mean
family_status                         
Не женат / не замужем   2810  0.097509
в разводе               1195  0.071130
вдовец / вдова           959  0.065693
гражданский брак        4163  0.093202
женат / замужем        12344  0.075421


##### Вывод 2:

Средняя доля кредитов с задолженностью по выборке - 8,1%.

Доля кредитов с задолженностью выше средней по выборке среди заемщиков, имеющие следующие семейные статусы:

- Не женат / не замужем - 9,75%;
- гражданский брак - 9,32%.

Доля кредитов с задолженностью ниже средней по выборке среди заемщиков, имеющие следующие семейные статусы:

- в разводе - 7,11%;
- вдовец / вдова - 6,56%;
- женат / замужем - 7,54%.

Можно сделать следующие выводы:
- заемщики, состоящие в стабильных отношениях (женат / замужем) с большей гарантией вернут кредит, чем заемщики, живущие в гражданском браке и не состоящие в браке;
- заемщики из групп "в разводе" и "вдовец/вдова" так же возвращают кредит лучше, чем в среднем по выборке; возможно это связано с тем, что в данные группы входят люди старшего возраста, имеющие более стабильное финансовое положение.
- самыми ненадежными заемщиками являются группы "не женат / не замужем" и "гражданский брак". Можно сделать предположение, что в эти группы входят заемщики более молодого возраста, имеющие нестабильное финансовое положение.

**Т.к. в выводах появилась гипотеза о связи возраста заемщика с его финансовым поведением, поверим эиу гипотезу.**

In [32]:
print(df.groupby('family_status')['dob_years'].agg({'count', 'median'})) #рассчитаем медианный возраст заемщиков в группах по семеному статусу

                       median  count
family_status                       
Не женат / не замужем      36   2810
в разводе                  46   1195
вдовец / вдова             58    959
гражданский брак           41   4163
женат / замужем            43  12344


Расчет медианного возраста подтвердил гипотезу, описанную выше: заемщики в группах с более высоким риском задолженности по кредиту "не женат / не замужем" и "гражданский брак" моложе, чем заемщики в группах "женат / замужем", "в разводе" и "вдовец/вдова".

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на данный вопрос сравним среднее значение для столбца debt (оно отображает долю кредитов с задолженностью по всей выборке) со средними значениями debt для групп заемщиков, относящихся к разным категориям по уровню дохода. Так же посчитаем, количество заемщиков в каждой группе.

In [33]:
print(average_debt)
print(df.groupby('total_income_category')['debt'].agg({'count', 'mean', }))

0.08108611615667645
                       count      mean
total_income_category                 
A                         25  0.080000
B                       5042  0.070607
C                      16032  0.084830
D                        350  0.060000
E                         22  0.090909


##### Вывод 3:

При сравнении доли кредитов с задолженностью в разных категориях по уровню дохода можно сделать вывод, что риск возникновения задолженности не зависит от уровня дохода, т.к. для категории А (с самым высоким доходом) он сотавляет 8%, а для категории D (препоследняя по уровню дохода), 6%.

Однако, т.к. общее число заемщиков в категориях A, B и E составляет незначительную часть выборки, основной вывод сформулируем на сравнии самых многочисленных категорий B и С. Для категории B доля кредитов с задолженностью равна 7%, что ниже средней на 1,1%. В категории С доля кредитов с задолженностью 8,48%, что выше средней на 0,38%. Исходя из этого можно сделать вывод, что риск задолженности по кредиту обратно пропорционален уровню доходов заемщика.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Для ответа на данный вопрос сравним среднее значение для столбца debt (оно отображает долю кредитов с задолженностью по всей выборке) со средними значениями debt для групп заемщиков, относящихся к разным категориям по цели кредита. Так же посчитаем, количество заемщиков в каждой группе.

In [34]:
print(average_debt)
print(df.groupby('purpose_category')['debt'].agg({'count', 'mean', }))

0.08108611615667645
                          count      mean
purpose_category                         
операции с автомобилем     4308  0.093547
операции с недвижимостью  10814  0.072314
получение образования      4014  0.092177
проведение свадьбы         2335  0.079657


Цели получения кредита оказывают влияние на его возврат в срок - по кредитам на цели "операции с недвижимостью" и "проведение свадьбы" задолженность возникает реже, чем в среднем по выборке; по кредитам на цели "операции с автомобилем" и "получение образования" задолженность возникает чаще, чем в среднем по выборке.

## Общий вывод:

**Мы дали ответы на следующие вопросы:**


1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

Чем больше детей в семье заемщика, тем выше риск невозврата кредиты в срок.


2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Заемщики, состоящие в стабильных отношениях (женат / замужем) с большей гарантией вернут кредит, чем заемщики, живущие в гражданском браке и не состоящие в браке;
заемщики из групп "в разводе" и "вдовец/вдова" так же возвращают кредит лучше, чем в среднем по выборке.


3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Риск задолженности по кредиту обратно пропорционален уровню доходов заемщика.


4. Как разные цели кредита влияют на его возврат в срок?

Цели получения кредита оказывают влияние на его возврат в срок - по кредитам на цели "операции с недвижимостью" и "проведение свадьбы" задолженность возникает реже, чем в среднем по выборке; по кредитам на цели "операции с автомобилем" и "получение образования" задолженность возникает чаще, чем в среднем по выборке.

**В процессе исследования были обнаружены проблемы со сбором данных:**
1. Отрицательные и слишком большие значения с столбце children.
2. Отрицательные и слишком большие значения с столбце days_employed.


Необходимо обратить на это внимание сотрудников, занимающихся сбором данных. Можно рекомендовать при заполнении анкет заемщиков поставить проверку в данных полях на отрицательные значения и значения, превышающие максимальный лимит. Так же можно рекомендовать сбор данных о стаже вести в годах, т.к. эта метрика не менее информативная, но значительно проще к заполнению.